# Get Item LOINCs from Q

1.  start with list of panel LOINCs
1. fetch Q from from LOINC
1. extract item codes
1. build VS
1. save etc.

In [1]:
from json import dumps, loads
import yaml
from pathlib import Path
from survey_obs import survey_obs
from requests import get,post
import panel_loincs
from stringcase import snakecase, titlecase, pascalcase
from datetime import datetime, date
from IPython.display import display as Display, HTML

In [2]:
out_dir = Path.cwd() / 'output'
#out_dir = r'/Users/ehaas/Documents/FHIR/Healthedata1-Sandbox/input/examples-yaml/'
panel_loincs.clintests

['34534-8',
 '92846-5',
 '18754-2',
 '34552-0',
 '18750-0',
 '78950-3',
 '82654-5',
 '59282-4',
 '46213-5',
 '78895-0',
 '64098-7',
 '83311-1',
 '97100-2',
 '28016-4',
 '51393-7',
 '97101-0',
 '18746-8',
 '18753-4',
 '18751-8',
 '11523-8',
 '98604-2',
 '18749-2',
 '29754-9',
 '92050-4',
 '98495-5',
 '79894-2',
 '95298-6',
 '95318-2',
 '96227-4',
 '79898-3',
 '57119-0',
 '96246-4',
 '79895-9',
 '79896-7',
 '98497-1',
 '77199-8',
 '89015-2',
 '18744-3',
 '81459-0',
 '90568-7',
 '81458-2',
 '80525-9',
 '45033-8']


### Validate and Write Functions

In [3]:
fhir_test_server = 'http://test.fhir.org/r4/'
#fhir_test_server = 'http://hapi.fhir.org/baseR4'
#fhir_test_server = 'http://wildfhir4.aegis.net/fhir4-0-1'

headers = {
'Accept':'application/fhir+json',
'Content-Type':'application/fhir+json'
}

# profile = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient' # The official URL for this profile is: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient

    
def validate(r):

    params = dict(
      # profile = profile
    )
    
    r = post(f'{fhir_test_server}/{r.resource_type}/$validate', headers = headers, data =r.json())
    print(r.status_code)
    # view  output
    display(HTML( (r.json()["text"]["div"])))



def write_out(fhir_obj):
    out_path = Path(out_dir) / f'{fhir_obj.resource_type}-{fhir_obj.id}.yml'   
    # modified the serialize to fix the date time format in JSON
    out_path.write_text(fhir_obj)
    print(f'writing to {out_path}....')

### Creat ValueSet ( all Manual )

In [4]:
from fhir.resources.questionnaire import Questionnaire as Q
from fhir.resources.valueset import ValueSet as V
my_vs = V(status='active')
my_vs.id = 'us-core-clinical-test-codes'
my_vs.url = f'http://hl7.org/fhir/us/core/ValueSet/{my_vs.id}'
my_vs.title = f'{titlecase(my_vs.id)} ValueSet'
my_vs.title = my_vs.title.replace('Us ','US ')
my_vs.name = my_vs.title.replace(' ','')
my_vs.status = "active"
my_vs.date = date.today()
my_vs.description = 'Starter set of clinical test LOINC codes identified in Appendix B in U.S. Core Data for Interoperability (USCDI) Task Force 2021 HITAC Phase 3 Recommendations Report Letter'
my_vs.copyright = 'LOINC\xAE (http://loinc.org). LOINC is copyright 1995-2020, Regenstrief Institute, Inc. and the Logical Observation Identifiers Names and Codes (LOINC) Committee and is available at no cost under the license at http://loinc.org/license. LOINC® is a registered United States trademark of Regenstrief Institute, Inc'
my_vs.compose = panel_loincs.vs_include

print(my_vs.yaml(indent=True))

resourceType: ValueSet
id: us-core-clinical-test-codes
url: http://hl7.org/fhir/us/core/ValueSet/us-core-clinical-test-codes
name: USCoreClinicalTestCodesValueSet
title: US Core Clinical Test Codes ValueSet
status: active
date: 2022-04-08
description: a starter set of clinical test LOINC codes identified in Appendix B in
  U.S. Core Data for Interoperability (USCDI) Task Force 2021 HITAC Phase 3 Recommendations
  Report Letter
copyright: "LOINC\xAE (http://loinc.org). LOINC is copyright 1995-2020, Regenstrief
  Institute, Inc. and the Logical Observation Identifiers Names and Codes (LOINC)
  Committee and is available at no cost under the license at http://loinc.org/license.
  LOINC\xAE is a registered United States trademark of Regenstrief Institute, Inc"
compose:
  include:
  - system: http://loinc.org



### Fetch Questionnaire Instance from LOINC

In [5]:
headers = {
    'Accept':'application/fhir+json',
    'Content-Type':'application/fhir+json'
    } 
fhir_server = 'https://fhir.loinc.org/' #/Questionnaire/?url=http://loinc.org/q/24331-1'
auth=('ehaas', 'loinc')
    
    
def fetch(my_loinc):

    params = dict(
      url = f'http://loinc.org/q/{my_loinc}'
        )   
    r = get(f'{fhir_server}/Questionnaire/', params = params, headers = headers, auth=auth)
    if r.status_code == 200:
        return r.json()
    else:
        print(f'http response = {r.status_code}')

In [6]:
concepts=[]
for p in panel_loincs.clintests: 
    # print(p)
    concepts.append({'code':p})#,'display':my_q.code[0].display})
    my_obj = fetch(p)
    try:
        q_obj = my_obj['entry'][0]['resource']
    except KeyError:
        continue
        #print(dumps(my_obj, indent=2))
    my_q = Q.parse_obj(q_obj)
    print('.', end='')
    #print (my_q.yaml())
    #print(f'LOINC = {my_q.code[0].code} ({my_q.code[0].display})')
    try:
     for i in my_q.item:
        if i.code[0].system == "http://loinc.org":
          print('.', end='')
          #print(f'  LOINC = {i.code[0].code}')
          concepts.append({'code':i.code[0].code,})#'display':i.code[0].display})
    except  (KeyError, TypeError):
        pass
#print(concepts) 
my_vs.compose.include[0].concept = concepts
print()
print(my_vs.yaml())

...........................................................................................................................................................................................................................................................................................................................................................................................................................................................
resourceType: ValueSet
id: us-core-clinical-test-codes
url: http://hl7.org/fhir/us/core/ValueSet/us-core-clinical-test-codes
name: USCoreClinicalTestCodesValueSet
title: US Core Clinical Test Codes ValueSet
status: active
date: 2022-04-08
description: a starter set of clinical test LOINC codes identified in Appendix B in
  U.S. Core Data for Interoperability (USCDI) Task Force 2021 HITAC Phase 3 Recommendations
  Report Letter
copyright: "LOINC\xAE (http://loinc.org). LOINC is copyright 1995-2020, Regenstrief
  Institute, Inc. and the Logical Observa

### Validate

In [7]:
validate(my_vs)

500


### Save

In [8]:
write_out(my_vs)

writing to /Users/ehaas/Documents/Python/Jupyter/MyNotebooks/QR-OBs-DE/output/ValueSet-us-core-clinical-test-codes.yml....
